In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=-7DDn65i10SL3QtV_tRcVwa9CE5WOYmK67qFmf6ftJY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgFn6mqqeyYFz_fnrX3rqIiCysVnF0Zjus02Mi4ltaBRQSuoNxRr8w

Successfully saved authorization token.


In [25]:
STATEFP = 12
STATENM = "FLORIDA"

In [8]:
state = ee.FeatureCollection("TIGER/2016/States").filter(ee.Filter.eq('STATEFP', str(STATEFP)))
counties = ee.FeatureCollection("TIGER/2016/Counties").filter(ee.Filter.eq('STATEFP', str(STATEFP)))

In [10]:
elevation = ee.Image('USGS/NED').select('elevation')
slope = ee.Terrain.slope(elevation).clip(state)

In [12]:
landcover2011 = ee.Image("USGS/NLCD/NLCD2011").select('landcover').clip(state) 
landcover2016 = ee.Image("USGS/NLCD/NLCD2016").select('landcover').clip(state)
  
impervious2011 = ee.Image("USGS/NLCD/NLCD2011").select('impervious').clip(state)
impervious2016 = ee.Image("USGS/NLCD/NLCD2016").select('impervious').clip(state)

In [15]:
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']

def maskL8sr(image):
  
  #stock method to remove clouds
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)

  #perform adjustments to make the numbers more sensical
  return image.select(opticalBands).divide(10000).addBands(
      image.select(thermalBands).divide(10).clamp(273.15, 373.15).subtract(273.15).divide(100)).updateMask(mask)

In [16]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2020-01-01', '2020-12-31').map(maskL8sr);

In [22]:
def ndvier(image):
  ndvied = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvied);

In [23]:
ndvied = l8.map(ndvier)
ndviest = ndvied.qualityMosaic('NDVI').select('NDVI').clip(state)

In [24]:
composite = landcover2016.addBands([landcover2011, impervious2016, impervious2011, slope, ndviest])
composite = composite.float()

In [30]:
countylist = counties.toList(counties.size())

In [39]:
for g in range(counties.size().getInfo()):  
  COUNTYNM = ee.Feature(countylist.get(g)).get("NAME").getInfo()

  desc =  str(STATENM) + '_comp_' + str(COUNTYNM)
  print(desc)
  task = ee.batch.Export.image.toDrive(
    image = composite,
    folder = STATENM,
    fileNamePrefix = desc,
    description = desc, 
    region = ee.Feature(countylist.get(g)).geometry(), 
    scale = 30, 
  )

  task.start()

FLORIDA_comp_Okaloosa
FLORIDA_comp_Walton
FLORIDA_comp_Bay
FLORIDA_comp_Gulf
FLORIDA_comp_Pinellas
FLORIDA_comp_Pasco
FLORIDA_comp_Hernando
FLORIDA_comp_Hillsborough
FLORIDA_comp_Monroe
FLORIDA_comp_Hendry
FLORIDA_comp_Highlands
FLORIDA_comp_Polk
FLORIDA_comp_Hardee
FLORIDA_comp_Brevard
FLORIDA_comp_Marion
FLORIDA_comp_Citrus
FLORIDA_comp_Lee
FLORIDA_comp_Collier
FLORIDA_comp_Alachua
FLORIDA_comp_Columbia
FLORIDA_comp_Gilchrist
FLORIDA_comp_St. Johns
FLORIDA_comp_Duval
FLORIDA_comp_Nassau
FLORIDA_comp_Baker
FLORIDA_comp_Clay
FLORIDA_comp_Putnam
FLORIDA_comp_Miami-Dade
FLORIDA_comp_Broward
FLORIDA_comp_Palm Beach
FLORIDA_comp_Okeechobee
FLORIDA_comp_Martin
FLORIDA_comp_Indian River
FLORIDA_comp_St. Lucie
FLORIDA_comp_Manatee
FLORIDA_comp_Sarasota
FLORIDA_comp_Charlotte
FLORIDA_comp_DeSoto
FLORIDA_comp_Sumter
FLORIDA_comp_Lake
FLORIDA_comp_Osceola
FLORIDA_comp_Orange
FLORIDA_comp_Volusia
FLORIDA_comp_Flagler
FLORIDA_comp_Seminole
FLORIDA_comp_Santa Rosa
FLORIDA_comp_Escambia
FLORIDA_comp